## Required libraries

In [ ]:
!pip install praw
!pip install PySimpleGUI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 KB 2.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 KB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import praw
import PySimpleGUI as sg
import re

### Function to locate the text file with all the reddit sites  

In [ ]:
def reddit_txt():
    while True:
        file = sg.popup_get_file("Select Plane-Data csv file location",
                                 title='My File Browser',
                                 file_types=(("ALL txt Files", "*.txt"),))
        if file != (''):
            break
        sg.popup_error(' Please enter a file path ')
    return file

### Function to get all the urls from the text file into a list

In [ ]:

def getUrls(txtfile):
    urls=[]
    with open(txtfile) as Fileobj:
        for lines in Fileobj:
            urls.append(lines)
    return urls


### Setting reddit api credentials required to be used to use the api

In [ ]:
def getPraw():
 return praw.Reddit(user_agent="Comment Extraction (by /u/rddit_scrapper)",
                     client_id="iD5TP-sX0fWQjXLskZUAsw", 
                     client_secret="3thlw2TxzNqrs0XsxUzSB7ReX47Cbg",
                     check_for_async=False)

### Function uses regex to look for keywords within the url to indicate the type of post and returns its corresponding type

In [ ]:
def trialtype(url):
    if re.search('(?i)cancer', url):
        return 'cancer'
    elif re.search('(?i)(corona)|(covid)|(novavax)', url):
        return 'covid'
    else:
        return 'other'

### The main function to get all the comments of all the reddit sites 

In [ ]:
def getComments(reddit,urls):
    
    #This list will a nest list of each url's comments 
    allcommentslist = []
    
    #loops through the urls list 
    for url in urls:
        
        #This list will hold all the comments of the current url only
        url_list= []
        
        #Appending the type of post
        url_list.append(trialtype(url))
       
        #Setting the url 
        submission = reddit.submission(url=url)
        
        #All 'More Comments' objects will be replaced until there are none left, 
        #as long as they satisfy the threshold
        submission.comments.replace_more(limit=None)
        
        #iterating through a list of all the comments and appends it to a list
        for comment in submission.comments.list():
            url_list.append(comment.body)
        
        #appending the url list  to the main list to hold all urls' comments
        allcommentslist.append(url_list)
        
    return allcommentslist



In [ ]:
def CreateDataframe(all_comments):
    #creates a empty dataframe that will concatinate all urls comments which its type
    df = pd.DataFrame([],columns=['comments','type'])
    
    for comments in all_comments:
        #taking out the first value which holds the site's type
        comment_type = comments.pop(0)
        #Creating a dataframe to hold the comments each loop with the column head as comments
        x= pd.DataFrame(comments,columns =['comments'])
        #creates a column type to label all the comments with the site's type
        x['type'] = comment_type
        #concatinate the dataframe with each iteration to hold all the comments labeled with its type
        df = pd.concat([df,x])
       
    return df



### Function to remove any comments that was deleted or removed on the site

In [ ]:

def cleanDataFrame(df):
    indexNames = df[(df.comments == '[removed]') | (df.comments == '[deleted]')].index
    df.drop(indexNames, inplace=True)
    
    return df

### Function to export the datafram to a csv

In [ ]:
def export_to_csv(df):
    #call a GUI to select output folder
    destfolder = sg.PopupGetFolder('Please select destination folder for extracted features')
    destfolder = destfolder.replace('/','\\\\')
    destfolder += '\\\\'
    df.to_csv(destfolder + 'reddit.csv',index=False)

### Main function 

In [ ]:
def main():
    txtfile = reddit_txt()
    reddit = getPraw()
    urls = getUrls(txtfile)
    all_comments = getComments(reddit,urls)    
    df = CreateDataframe(all_comments)
    df = cleanDataFrame(df)
    export_to_csv(df)



In [ ]:
# Calling main function 
if __name__=="__main__": 
    main()
